## Pre-embedding Notes 

In [1]:
import torch
torch.cuda.is_available()


/opt/conda/Anaconda3/envs/IRTS/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [2]:
import transformers
import pandas as pd 
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
import pickle



/opt/conda/Anaconda3/envs/kanbefair/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/Anaconda3/envs/kanbefair/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## In-Hospital-Mortality

In [ ]:

## Preprocessing the MIMIC-III
for i in ['train', 'test']:
    data_path = f'/home/nathan/Missing_Modality/Data/mimic-iii-processed/in-hospital-mortality/{i}p2x_data.pkl'
    with open(data_path, 'rb') as file:
        data = pickle.load(file)
        
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    for idx, patient in enumerate(data):
        if 'text_data' in patient:
            inputs = tokenizer(patient['text_data'], padding=True, return_tensors="pt", max_length=256, truncation=True)
            with torch.no_grad():
                outputs = model(**inputs)
                last_hidden_state = outputs.last_hidden_state
                last_hidden_state = last_hidden_state.mean(dim=0)
            
        else:
            last_hidden_state = torch.zeros((1,1,1))
        
        data[idx]['text_embedding'] = last_hidden_state.squeeze(0)

    with open(f"/home/nathan/Missing_Modality/Data/mimic-iii-processed/in-hospital-mortality/{i}_embedding.pkl", "wb") as save_obj:
        data_obj = pickle.dump(data, save_obj)

### Length-of-Stay

In [3]:

## Preprocessing the MIMIC-III
for i in ['train', 'test']:
    data_path = f'/home/nathan/Missing_Modality/Data/mimic-iii-processed/length-of-stay/{i}p2x_data.pkl'
    with open(data_path, 'rb') as file:
        data = pickle.load(file)
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    for idx, patient in enumerate(data):
        if 'text_data' in patient:
            inputs = tokenizer(patient['text_data'], padding=True, return_tensors="pt", max_length=256, truncation=True)
            with torch.no_grad():
                outputs = model(**inputs)
                last_hidden_state = outputs.last_hidden_state
                last_hidden_state = last_hidden_state.mean(dim=0)
        else:
            last_hidden_state = torch.zeros((1,1,1))
        
        data[idx]['text_embedding'] = last_hidden_state.squeeze(0)
        # Consider this as binary classification task too
        # less than 3 days --> 0
        # more than 3 days --> 1
        thresh = 24 * 3 
        data[idx]['label'] = int(data[idx]['label'] >= thresh)
        
    with open(f"/home/nathan/Missing_Modality/Data/mimic-iii-processed/length-of-stay/{i}_embedding.pkl", "wb") as save_obj:
        data_obj = pickle.dump(data, save_obj)

## Phenotyping

In [4]:

## Preprocessing the MIMIC-III
for i in ['train', 'test']:
    data_path = f'/home/nathan/Missing_Modality/Data/mimic-iii-processed/phenotyping/{i}p2x_data.pkl'
    with open(data_path, 'rb') as file:
        data = pickle.load(file)
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    for idx, patient in enumerate(data):
        if 'text_data' in patient:
            inputs = tokenizer(patient['text_data'], padding=True, return_tensors="pt", max_length=256, truncation=True)
            with torch.no_grad():
                outputs = model(**inputs)
                last_hidden_state = outputs.last_hidden_state
                last_hidden_state = last_hidden_state.mean(dim=0)
        else:
            last_hidden_state = torch.zeros((1,1,1))
        
        data[idx]['text_embedding'] = last_hidden_state.squeeze(0)
        
    with open(f"/home/nathan/Missing_Modality/Data/mimic-iii-processed/phenotyping/{i}_embedding.pkl", "wb") as save_obj:
        data_obj = pickle.dump(data, save_obj)